In [1]:
import os
import time
from datetime import datetime

import cv2
import yt_dlp
from ultralytics import YOLO
from collections import defaultdict
import supervision as sv


In [2]:
video_url = "https://www.youtube.com/watch?v=0Pg3S6s76IE"  # Youtube URL northbound


In [3]:
def get_stream_url(youtube_url):
    ydl_opts = {
        'quiet': True,
        'skip_download': True,
        'no_warnings': True,
        'force_generic_extractor': False,
        'format': 'best[ext=mp4][protocol^=http]/best'
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(youtube_url, download=False)
        formats = info.get('formats', [info])
        # prefer video-only best mp4 OR best overall
        for f in reversed(formats):
            if f.get('vcodec', '') != 'none' and f.get('acodec', '') == 'none' and 'url' in f:
                return f['url']
        # fallback: best available format
        return formats[-1]['url']

In [ ]:
# Load the YOLO model
model = YOLO('yolo11l.pt')

class_list = model.names 

# Dictionary to store object counts by class
class_counts = defaultdict(int)

# Dictionary to keep track of object IDs that have crossed the line
crossed_ids = set()

# the line
line_y_red = 700  # Red line position


def capture_frames(stream_url):

    # Open the video file
    cap = cv2.VideoCapture(stream_url)
    #i = 0

    while True:
        
        ret, frame = cap.read() # read each frame one by one
        
        if not ret: #when there are no more frames, break
            # Release resources on way out of while loop
            print("on way to first break")
            cap.release()
            cv2.destroyAllWindows()
            break

        
        # Run YOLO tracking on the frame
        results = model.track(frame, classes = [1,2,3,5,6,7]) #persist=True was on before, but it would make the loop fail...
        print(results)

        # Ensure results are not empty
        if results[0].boxes.data is not None:
            # Get the detected boxes, their class indices, and track IDs
            boxes = results[0].boxes.xyxy.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()
            class_indices = results[0].boxes.cls.int().cpu().tolist()
            confidences = results[0].boxes.conf.cpu()

            cv2.line(frame, (550, line_y_red), (1250, line_y_red), (0, 0, 255), 2)
            #cv2.putText(frame, 'Red Line', (690, line_y_red - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        
            # Loop through each detected object
            for box, track_id, class_idx, conf in zip(boxes, track_ids, class_indices, confidences):
                x1, y1, x2, y2 = map(int, box)
                cx = (x1 + x2) // 2  # Calculate the center point
                cy = (y1 + y2) // 2            

                class_name = class_list[class_idx]

                cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
                
                cv2.putText(frame, f"ID: {track_id} {class_name}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2) 

                # Check if the object has crossed the red line
                if cy > line_y_red and cx > 550 and cx < 1250 and track_id not in crossed_ids:
                    # Mark the object as crossed
                    crossed_ids.add(track_id)
                    class_counts[class_name] += 1

            # Display the counts on the frame
            y_offset = 30
            cv2.putText(frame, ('Crossed Vehicles: ' + str(len(crossed_ids))), (50, y_offset),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            for class_name, count in class_counts.items():
                y_offset += 30
                cv2.putText(frame, f"{class_name}: {count}", (50, y_offset),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        ### Save frame    
        # prep timestamp for file name
        current_datetime = datetime.now().strftime("%Y-%m-%d %H-%M-%S")
        # Save as PNG
        filename =r'c:\Users\Nineveh.OConnell\OneDrive - DOT OST\Documents\code\Border-Bottleneck-Management\support_scripts\captured_pngs'
        cv2.imwrite(f"{filename}/frame_{str(current_datetime)}.png", frame)
        print(f"Saved frame {current_datetime}")
        #i += 1
        #cap.set(1, 10) # skip frames, 1 corresponds to the propid index, 10 skips 10 frames I believe
        
        # Show the frame
        cv2.imshow("YOLO Object Tracking & Counting", frame)    
        
        # Exit loop if 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'): #removed:  
            # Release resources on way out of while loop
            print("on way to second break")
            cap.release()
            cv2.destroyAllWindows()
            break


if __name__ == "__main__":
    stream_url = get_stream_url(video_url)
    print("Stream URL:", stream_url)
    capture_frames(stream_url)

    


Stream URL: https://manifest.googlevideo.com/api/manifest/hls_playlist/expire/1750276168/ei/6MNSaKqMEu_ou_kPr6r3gAg/ip/152.122.255.5/id/0Pg3S6s76IE.2/itag/270/source/yt_live_broadcast/requiressl/yes/ratebypass/yes/live/1/sgovp/gir%3Dyes%3Bitag%3D137/rqh/1/hls_chunk_host/rr2---sn-vgqsrnzd.googlevideo.com/xpc/EgVo2aDSNQ%3D%3D/playlist_duration/30/manifest_duration/30/bui/AY1jyLOoGa3OFVi9uQIC5bYE_rUx0DLTPPMWj5AS5jCVesZQaWdZn19ePRM_64SWIaU8CcHjMhCEShkj/spc/l3OVKaS8V74b8WrSF-OrcCGaL-7FFPIS5IPSN3Q44xuca3HGrqlRHs87rbCyNQ/vprv/1/playlist_type/DVR/initcwndbps/1445000/met/1750254568,/mh/sg/mm/44/mn/sn-vgqsrnzd/ms/lva/mv/m/mvi/2/pl/16/rms/lva,lva/dover/13/pacing/0/short_key/1/keepalive/yes/fexp/51466698,51476175/mt/1750254047/sparams/expire,ei,ip,id,itag,source,requiressl,ratebypass,live,sgovp,rqh,xpc,playlist_duration,manifest_duration,bui,spc,vprv,playlist_type/sig/AJfQdSswRAIgTEBoetiZqlmHzcivCr7wyNpOEk37iz_Rru6l0apYhQICICmDlQXj7RozRS5-sEqYJPdbmuXadIrNyY_-4j93pmK2/lsparams/hls_chunk_host,initcw